<a href="https://colab.research.google.com/github/alexmoed/MastersProject_Submission/blob/main/scripts/Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#If using colab mount drive first
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


 @brief Pointcept training setup and workflow for ScanNet datasets

 Environment setup, dataset preprocessing, and training pipeline organization
 assisted by Claude AI (Anthropic). Specific assistance included:
 - Dataset class remapping from ScanNet 20 to 2-class (wall/floor) workflow
 - Module import handling and sys.path configuration for custom dataset classes
 - Training command organization and parameter setup
 - Environment variable configuration for CUDA and wandb
 - File copying and module reloading procedures for custom ScanNet3Dataset
 Multiple prompts used across extended conversation for troubleshooting
 installation issues, organizing training workflows, and coordinating
 semantic/instance segmentation training procedures.

 Base training framework and tools from:
 Pointcept Contributors (2023). Pointcept: A Codebase for Point Cloud Perception Research [online].
 [Accessed 2025]. Available from: "https://github.com/Pointcept/Pointcept".# Original Author: Xiaoyang Wu (xiaoyang.wu.cs@gmail.com)

INSTALLS

In [ ]:
#Numpy versions are a ongoing issue in this install it must be below 1.25 to work 1.24.and 1.24.3 are tested and work
#Uninstall numpy first
#Restart session!!
!pip uninstall -y numpy
!pip uninstall -y numpy  # Run twice

# Find any installs of numpy left
!find /usr/local/lib/python*/site-packages -name "numpy*" -type d

# Delete ALL numpy folders
!rm -rf /usr/local/lib/python*/site-packages/numpy*
!rm -rf /usr/local/lib/python*/dist-packages/numpy*
!rm -rf ~/.local/lib/python*/site-packages/numpy*

# Clean pip cache too
!pip cache purge

# Fresh install
!pip install numpy==1.24.0 --no-cache-dir

# Make sure the versions are correct and reload if not it needs to be 1.24.0
import numpy as np
if np.__version__ != "1.24.0":
   print(f"Wrong version: {np.__version__}")


print(f"Numpy version: {np.__version__}")
print(f"Numpy location: {np.__file__}")


Found existing installation: numpy 1.24.0
Uninstalling numpy-1.24.0:
  Successfully uninstalled numpy-1.24.0
find: ‘/usr/local/lib/python*/site-packages’: No such file or directory
Files removed: 0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 241.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 2.0.8 requires numpy>=1.24.4, but you have numpy 1.24.0 which is incompatible.
seaborn 0.13.2 requires numpy!=1.24.0,>=1.20, but you have numpy 1.24.0 which is incompatible.
albucore 0.0.24 requires numpy>=1.24.4, but you have numpy 1.24.0 which is incompatible.
jaxlib 0.5.3 requires numpy>=1.25, but you have numpy 1.24.0 which is incompatible.
pywavelets 1.9.0 requires numpy<3,>=1.25, but you have numpy 1.24.0 which is incompatible.
scipy 1.16.1 requires numpy<2.6,>=1.25.2, but you have numpy 1.24.0 which is incompatible.
ten

Numpy version: 1.24.0
Numpy location: /usr/local/lib/python3.11/dist-packages/numpy/__init__.py


In [ ]:
#Import universal dependencies that arent impacted by order
import os
import sys
import subprocess
import shutil
from pathlib import Path
from datetime import datetime
import gc
import json
import time
from collections import OrderedDict
from glob import glob



This is a verison cuda install that after running once that it saves the download and caches the install for speed. Also handles torch installs

---



In [ ]:
# first batch of installs installing cuda and pytorch older versions: This will take 10+ minutes
%cd /content/drive/MyDrive/Pointcept/Installs
exec(open('/content/drive/MyDrive/Pointcept/Installs/setup_cuda_torch.py').read())


/content/drive/MyDrive/Pointcept/Installs
CUDA INSTALLATION

1. Cleaning existing CUDA installations...
✓ Cleaned

2. Installing CUDA keyring...
✓ Using cached keyring
✓ Keyring installed

3. Installing CUDA Toolkit 11.8...
✓ CUDA 11.8 installed

4. Setting environment variables...
✓ Environment set

5. Verifying installation...

✅ CUDA installation complete!
Cached files in: /content/drive/MyDrive/colab_cache

PYTORCH INSTALLATION
PyTorch: 2.1.0+cu118
CUDA available: True
CUDA version: 11.8
NumPy: 1.24.0


In [ ]:
# Install pointops that is needed for Sonata decoder head. This you have to run twice

%cd Pointcept/libs/pointops
!pip install -v -e.
%cd /content/drive/MyDrive/Pointcept


/content/drive/MyDrive/Pointcept/Pointcept/libs/pointops
Using pip 24.1.2 from /usr/local/lib/python3.11/dist-packages/pip (python 3.11)
Obtaining file:///content/drive/MyDrive/Pointcept/Pointcept/libs/pointops
  Running command python setup.py egg_info
  running egg_info
  creating /tmp/pip-pip-egg-info-l9lu9k0y/pointops.egg-info
  writing /tmp/pip-pip-egg-info-l9lu9k0y/pointops.egg-info/PKG-INFO
  writing dependency_links to /tmp/pip-pip-egg-info-l9lu9k0y/pointops.egg-info/dependency_links.txt
  writing requirements to /tmp/pip-pip-egg-info-l9lu9k0y/pointops.egg-info/requires.txt
  writing top-level names to /tmp/pip-pip-egg-info-l9lu9k0y/pointops.egg-info/top_level.txt
  writing manifest file '/tmp/pip-pip-egg-info-l9lu9k0y/pointops.egg-info/SOURCES.txt'
  /usr/local/lib/python3.11/dist-packages/torch/utils/cpp_extension.py:502: UserWarning: Attempted to use ninja as the BuildExtension backend but we could not find ninja.. Falling back to using the slow distutils backend.
    warnin

In [ ]:
 #More installs and configs required
 %cd /content/drive/MyDrive/Pointcept/Installs
 exec(open('/content/drive/MyDrive/Pointcept/Installs/setup_build_env.py').read())

/content/drive/MyDrive/Pointcept/Installs
2.1.0+cu118
NumPy: 1.24.0
NumPy: 1.24.0
CHECKING ENVIRONMENT
✓ GPU Available: NVIDIA A100-SXM4-40GB
✓ CUDA Version: 11.8
✓ PyTorch Version: 2.1.0+cu118

✓ CUDA Version for installations: cu118

FIXING NUMPY COMPATIBILITY
✓ NumPy 1.24.0 compatibility fixes applied

INSTALLING DEPENDENCIES


<string>:93: FutureWarning: In the future `np.bool` will be defined as the corresponding NumPy scalar.  (This may have returned Python scalars in past versions.
<string>:93: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.  (This may have returned Python scalars in past versions.
<string>:93: FutureWarning: In the future `np.str` will be defined as the corresponding NumPy scalar.  (This may have returned Python scalars in past versions.


Installing basic dependencies...

Installing PyTorch Geometric dependencies...
Installing torch-scatter...
Installing torch-sparse...
Installing torch-cluster...
Installing torch-geometric...

Installing spconv...

CLONING POINTCEPT
✓ Cloned Pointcept
✓ GPU: NVIDIA A100-SXM4-40GB, Using CUDA architecture: 8.0

BUILDING POINTOPS
Building pointops with pip...
✓ pointops imported successfully

BUILDING POINTGROUP_OPS
Building pointgroup_ops with pip...
✓ pointgroup_ops imported successfully

FINAL VERIFICATION
✓ torch (2.1.0+cu118)
✓ numpy (1.24.0)
✓ sklearn (1.3.2)
✓ torch_cluster (installed)
✓ pointops (installed)
✓ pointgroup_ops (installed)
✓ spconv (2.3.8)

✅ Installation completed successfully!

To use Pointcept, add these lines at the start of your code:
```python
import sys
sys.path.insert(0, '/content/Pointcept')
sys.path.insert(0, '/content/Pointcept/libs/pointops')
sys.path.insert(0, '/content/Pointcept/libs/pointgroup_ops')
```

QUICK FUNCTIONALITY TEST
✓ Created test tensor o

In [ ]:
#Load and impliment pointops patches and wrapper
%cd /content/drive/MyDrive/Pointcept/Installs
exec(open('/content/drive/MyDrive/Pointcept/Installs/fix_ballquery_wrapper.py').read())

/content/drive/MyDrive/Pointcept/Installs
FIXING BALLQUERY WRAPPER AFTER TORCH_GEOMETRIC INSTALL

1. CHECKING ENVIRONMENT AND DEPENDENCIES
----------------------------------------
  Python version: 3.11.13
  ✓ Python version OK
  PyTorch version: 2.1.0+cu118
  CUDA available: True
  CUDA version: 11.8
  GPU: NVIDIA A100-SXM4-40GB
  PyTorch built with CUDA: 11.8

  Checking core dependencies:
  ✓ numpy
  ✓ plotly
  ✓ trimesh
  ✓ psutil

  ✓ Pointcept found at: /content/Pointcept

  Checking pointgroup_ops:
  ✓ pointgroup_ops found at: /content/drive/MyDrive/Pointcept/libs/pointgroup_ops/pointgroup_ops.cpython-311-x86_64-linux-gnu.so
  ⚠ WARNING: Loading .so file directly
  ✓ Has ballquery_batch_p: <class 'builtin_function_or_method'>
  ✓ pointgroup_ops_cuda found at: /usr/local/lib/python3.11/dist-packages/pointgroup_ops_cuda.cpython-311-x86_64-linux-gnu.so
  ✓ pointops installed (optional)


2. FIXING MODEL IMPORTS
----------------------------------------

  Processing point_group_v1m2

Training:

In [ ]:
#Required to run any training methods below

torch_lib_path = os.path.join(os.path.dirname(torch.__file__), 'lib')
os.environ['LD_LIBRARY_PATH'] = f"{torch_lib_path}:{os.environ.get('LD_LIBRARY_PATH', '')}"

# Disable wandb for training
os.environ['WANDB_MODE'] = 'disabled'
#set the path
%cd /content/drive/MyDrive/Pointcept

/content/drive/MyDrive/Pointcept


# Semantic Segementation training examples:

In [ ]:
# Make sure it works with scannet 20 dataset
!sed -i '/from .scannet import ScanNet3Dataset/d' /content/Pointcept/pointcept/datasets/__init__.py

In [ ]:


#Full scannet 20 semantic scene training
!python tools/train.py \
    --config-file /content/drive/MyDrive/Pointcept/configs/sonata/semseg-sonata-v1m1-0c-scannet-ft-fast.py \
    --num-gpus 1 \
    --options weight=/content/drive/MyDrive/sonata/Checkpoint/pretrain-sonata-v1m1-0-base.pth \
    save_path=exp/sonata/semseg-sonata-v1m1-0c-scannet-ft-_full_scene_v004


Number of Epochs: 800
✅ Loaded config semseg-sonata-v1m1-0c-scannet-ft-fast-a100.py optimized for A100 40GB
[2025-08-15 14:33:32,417 INFO train.py line 136 17551] => Loading config ...
[2025-08-15 14:33:32,417 INFO train.py line 138 17551] Save path: exp/sonata/semseg-sonata-v1m1-0c-scannet-ft-_full_scene_v004
[2025-08-15 14:33:33,051 INFO train.py line 139 17551] Config:
weight = '/content/drive/MyDrive/sonata/Checkpoint/pretrain-sonata-v1m1-0-base.pth'
resume = False
evaluate = False
test_only = False
seed = 31852439
save_path = 'exp/sonata/semseg-sonata-v1m1-0c-scannet-ft-_full_scene_v004'
num_worker = 2
batch_size = 4
batch_size_val = None
batch_size_test = None
epoch = 800
eval_epoch = 800
clip_grad = 3.0
sync_bn = False
enable_amp = True
amp_dtype = 'float16'
empty_cache = False
empty_cache_per_epoch = True
find_unused_parameters = False
enable_wandb = True
wandb_project = 'pointcept'
wandb_key = None
mix_prob = 0.8
param_dicts = [dict(keyword='block', lr=0.0002)]
hooks = [
    d

# Floor and wall only

Remap dataset ScanNet's 20 classes down to just 2 classes saved out npy files (wall and floor)

In [ ]:

data_root = "/content/drive/MyDrive/Scannet_dataset"

# Process train, val, and test splits
for split in ["train", "val", "test"]:
    split_path = os.path.join(data_root, split)

    if not os.path.exists(split_path):
        print(f"\nSkipping {split} split - not found")
        continue

    print(f"\nProcessing {split} split...")

    #Find all label files (using segment20.npy for 20-class labels)
    label_files = glob(os.path.join(data_root, split, "*", "segment20.npy"))

    for label_file in label_files:
        # Load labels
        labels = np.load(label_file)

        #Creating new labels: wall=0, floor=1, everything else=-1
        new_labels = np.full_like(labels, -1)
        new_labels[labels == 0] = 0  # wall
        new_labels[labels == 1] = 1  # floor

        #Save as a NEW file
        new_file = label_file.replace("segment20.npy", "segment20_2class.npy")
        np.save(new_file, new_labels)

        print(f"Processed: {os.path.basename(os.path.dirname(label_file))} -> {os.path.basename(new_file)}")

    print(f"Completed {split} split - {len(label_files)} scenes")

print("\n Done! Now update your config to use 'segment20_2class.npy' instead of 'segment20.npy'")


Processing train split...
Processed: scene0107_00 -> segment20_2class.npy
Processed: scene0110_02 -> segment20_2class.npy
Processed: scene0112_00 -> segment20_2class.npy
Processed: scene0106_00 -> segment20_2class.npy
Processed: scene0106_01 -> segment20_2class.npy
Processed: scene0112_01 -> segment20_2class.npy
Processed: scene0112_02 -> segment20_2class.npy
Processed: scene0111_01 -> segment20_2class.npy
Processed: scene0111_00 -> segment20_2class.npy
Processed: scene0113_01 -> segment20_2class.npy
Processed: scene0113_00 -> segment20_2class.npy
Processed: scene0116_00 -> segment20_2class.npy
Processed: scene0116_01 -> segment20_2class.npy
Processed: scene0114_00 -> segment20_2class.npy
Processed: scene0111_02 -> segment20_2class.npy
Processed: scene0114_02 -> segment20_2class.npy
Processed: scene0116_02 -> segment20_2class.npy
Processed: scene0115_00 -> segment20_2class.npy
Processed: scene0114_01 -> segment20_2class.npy
Processed: scene0115_01 -> segment20_2class.npy
Processed: sc

KeyboardInterrupt: 

This replace Pointcept's default scannet.py dataset script with a custom version that contains ScanNet3Dataset which is just the 3 classes floor ceiling and everything else, clearing any cached imports to ensure the new version is loaded.

In [ ]:


# First copy the file that has ScanNet3Dataset
shutil.copy(
   '/content/drive/MyDrive/Pointcept/pointcept/datasets/scannet.py',
   '/content/Pointcept/pointcept/datasets/scannet.py'
)
print("Copied scannet.py to the correct location")

# Check if ScanNet3Dataset is actually in the copied file
with open('/content/Pointcept/pointcept/datasets/scannet.py', 'r') as f:
   content = f.read()
   if 'class ScanNet3Dataset' in content:
       print("ScanNet3Dataset found in file")
       if '@DATASETS.register_module()' in content.split('class ScanNet3Dataset')[0][-100:]:
           print("Decorator is present")
   else:
       print("ScanNet3Dataset NOT found in file")

# Clear out any cached modules
if 'pointcept.datasets.scannet' in sys.modules:
   del sys.modules['pointcept.datasets.scannet']

for key in list(sys.modules.keys()):
   if key.startswith('pointcept'):
       del sys.modules[key]

# Add Pointcept to Python path
sys.path.insert(0, '/content/Pointcept')

# Now we can import ScanNet3Dataset
from pointcept.datasets.scannet import ScanNet3Dataset
print("Successfully imported ScanNet3Dataset")

# Double check the file exists
scannet_path = '/content/Pointcept/pointcept/datasets/scannet.py'
print(f"scannet.py exists: {os.path.exists(scannet_path)}")

Copied scannet.py to the correct location
ScanNet3Dataset found in file
Decorator is present
Successfully imported ScanNet3Dataset
scannet.py exists: True


Launch training for just wall and floor


In [ ]:
# Run semantic segmentation training for just walls and floors -1 for everything else
!python tools/train.py \
    --config-file configs/sonata/semseg-sonata-v1m1-0c-scannet-ft-_wall_ceiling_v001.py \
    --num-gpus 1 \
    --options weight=/content/drive/MyDrive/sonata/Checkpoint/pretrain-sonata-v1m1-0-base.pth \
    save_path=exp/scannet/semseg-sonata-v1m1-0c-scannet-ft-_wall_ceiling_v005




Number of Epochs: 500
✅ Loaded config semseg-sonata-v1m1-0c-scannet3-ft-fast-a100.py for 3-class segmentation
[2025-08-15 14:39:07,916 INFO train.py line 136 19107] => Loading config ...
[2025-08-15 14:39:07,917 INFO train.py line 138 19107] Save path: exp/scannet/semseg-sonata-v1m1-0c-scannet-ft-_wall_ceiling_v005
[2025-08-15 14:39:08,560 INFO train.py line 139 19107] Config:
weight = '/content/drive/MyDrive/sonata/Checkpoint/pretrain-sonata-v1m1-0-base.pth'
resume = False
evaluate = False
test_only = False
seed = 7298273
save_path = 'exp/scannet/semseg-sonata-v1m1-0c-scannet-ft-_wall_ceiling_v005'
num_worker = 2
batch_size = 3
batch_size_val = None
batch_size_test = None
epoch = 500
eval_epoch = 500
clip_grad = 3.0
sync_bn = False
enable_amp = True
amp_dtype = 'float16'
empty_cache = False
empty_cache_per_epoch = True
find_unused_parameters = False
enable_wandb = True
wandb_project = 'pointcept'
wandb_key = None
mix_prob = 0.8
param_dicts = [dict(keyword='block', lr=0.0001)]
hooks = 

# Instance Training

Scannet 200

In [ ]:
#Train on all 200 classes
!python tools/train.py \
  --config-file=/content/drive/MyDrive/Pointcept/Pointcept/configs/scannet200/insseg-pointgroup-v1m2-0-ptv3-base_1_scannet_200_v001.py \
  --num-gpus 1 \
  --options weight=/content/drive/MyDrive/sonata/Checkpoint/pretrain-sonata-v1m1-0-base.pth \
  start_epoch=1 \
  save_path=/content/drive/MyDrive/Pointcept/exp/scannet/insseg-pointgroup-sonata_config_200_1

[2025-08-15 14:42:27,428 INFO train.py line 136 20090] => Loading config ...
[2025-08-15 14:42:27,429 INFO train.py line 138 20090] Save path: /content/drive/MyDrive/Pointcept/exp/scannet/insseg-pointgroup-sonata_config_200_1
[2025-08-15 14:42:28,827 INFO train.py line 139 20090] Config:
CLASS_LABELS_200 = (
    'wall', 'chair', 'floor', 'table', 'door', 'couch', 'cabinet', 'shelf',
    'desk', 'office chair', 'bed', 'pillow', 'sink', 'picture', 'window',
    'toilet', 'bookshelf', 'monitor', 'curtain', 'book', 'armchair',
    'coffee table', 'box', 'refrigerator', 'lamp', 'kitchen cabinet', 'towel',
    'clothes', 'tv', 'nightstand', 'counter', 'dresser', 'stool', 'cushion',
    'plant', 'ceiling', 'bathtub', 'end table', 'dining table', 'keyboard',
    'bag', 'backpack', 'toilet paper', 'printer', 'tv stand', 'whiteboard',
    'blanket', 'shower curtain', 'trash can', 'closet', 'stairs', 'microwave',
    'stove', 'shoe', 'computer tower', 'bottle', 'bin', 'ottoman', 'bench',
    'boa

In [ ]:
# Make sure it works with scannet 20 dataset
!sed -i '/from .scannet import ScanNet3Dataset/d' /content/Pointcept/pointcept/datasets/__init__.py

Scannet 20

In [ ]:
#Train 20 Classes
!python tools/train.py \
  --config-file=/content/drive/MyDrive/Pointcept/Pointcept/configs/scannet/insseg-pointgroup-v1m2-0-ptv3-base_4.py \
  --num-gpus 1 \
  --options weight=/content/drive/MyDrive/sonata/Checkpoint/pretrain-sonata-v1m1-0-base.pth\
  start_epoch=1 \
  save_path=exp/scannet/insseg-pointgroup-sonata_config_4

[2025-08-15 14:44:03,075 INFO train.py line 136 20607] => Loading config ...
[2025-08-15 14:44:03,075 INFO train.py line 138 20607] Save path: exp/scannet/insseg-pointgroup-sonata_config_4
[2025-08-15 14:44:03,743 INFO train.py line 139 20607] Config:
CLASS_LABELS_200 = (
    'wall', 'chair', 'floor', 'table', 'door', 'couch', 'cabinet', 'shelf',
    'desk', 'office chair', 'bed', 'pillow', 'sink', 'picture', 'window',
    'toilet', 'bookshelf', 'monitor', 'curtain', 'book', 'armchair',
    'coffee table', 'box', 'refrigerator', 'lamp', 'kitchen cabinet', 'towel',
    'clothes', 'tv', 'nightstand', 'counter', 'dresser', 'stool', 'cushion',
    'plant', 'ceiling', 'bathtub', 'end table', 'dining table', 'keyboard',
    'bag', 'backpack', 'toilet paper', 'printer', 'tv stand', 'whiteboard',
    'blanket', 'shower curtain', 'trash can', 'closet', 'stairs', 'microwave',
    'stove', 'shoe', 'computer tower', 'bottle', 'bin', 'ottoman', 'bench',
    'board', 'washing machine', 'mirror', 'co